# Financial Statement Model Core Library - Basic Usage

This notebook demonstrates the basic features and usage patterns of the `fin_statement_model.core` library. We'll cover:

1. Creating and managing financial statement graphs
2. Adding financial statement items and calculation nodes
3. Working with metrics
4. Using adjustments for scenario analysis
5. Forecasting future values
6. Graph inspection and traversal


## 1. Setup and Basic Graph Creation

Let's start by importing the necessary components and creating a basic financial statement graph.


In [24]:
# Import the core components
from fin_statement_model.core import Graph
from fin_statement_model.core.adjustments import AdjustmentType
# Import metrics functionality
from fin_statement_model.core.metrics import metric_registry, calculate_metric, interpret_metric
# Import AdjustmentFilter for proper filtering
from fin_statement_model.core.adjustments import AdjustmentFilter

# Create a graph with historical periods
graph = Graph(periods=["2021", "2022", "2023"])

# Display the initial graph state
print(f"Graph created with periods: {graph.periods}")
print(f"Initial nodes: {list(graph.nodes.keys())}")
print(graph)


Graph created with periods: ['2021', '2022', '2023']
Initial nodes: []
<Graph(Total Nodes: 0, FS Items: 0, Calculations: 0, Dependencies: 0, Periods: ['2021', '2022', '2023'])>


## 2. Adding Financial Statement Items

Now let's add some basic financial statement items like Revenue, COGS, and Operating Expenses.


In [25]:
# Add revenue data
revenue_node = graph.add_financial_statement_item(
    "Revenue", 
    {"2021": 1000.0, "2022": 1200.0, "2023": 1400.0}
)

# Add cost of goods sold
cogs_node = graph.add_financial_statement_item(
    "COGS",
    {"2021": 600.0, "2022": 700.0, "2023": 800.0}
)

# Add operating expenses
opex_node = graph.add_financial_statement_item(
    "OperatingExpenses",
    {"2021": 200.0, "2022": 240.0, "2023": 280.0}
)

# Display current nodes
print("Financial statement items added:")
for node_name in graph.nodes:
    node = graph.nodes[node_name]
    print(f"  {node_name}: {node.values}")

# Calculate a specific value
print(f"\nRevenue for 2023: ${graph.calculate('Revenue', '2023'):,.2f}")


Financial statement items added:
  Revenue: {'2021': 1000.0, '2022': 1200.0, '2023': 1400.0}
  COGS: {'2021': 600.0, '2022': 700.0, '2023': 800.0}
  OperatingExpenses: {'2021': 200.0, '2022': 240.0, '2023': 280.0}

Revenue for 2023: $1,400.00


## 3. Creating Calculation Nodes

Let's create calculation nodes to derive metrics like Gross Profit, Operating Income, and profit margins.


In [26]:
# Create Gross Profit calculation (Revenue - COGS)
gross_profit = graph.add_calculation(
    name="GrossProfit",
    input_names=["Revenue", "COGS"],
    operation_type="subtraction"
)

# Create Operating Income (Gross Profit - Operating Expenses)
operating_income = graph.add_calculation(
    name="OperatingIncome",
    input_names=["GrossProfit", "OperatingExpenses"],
    operation_type="subtraction"
)

# Create Gross Profit Margin using formula
gross_margin = graph.add_calculation(
    name="GrossProfitMargin",
    input_names=["GrossProfit", "Revenue"],
    operation_type="formula",
    formula="(gross_profit / revenue) * 100",
    formula_variable_names=["gross_profit", "revenue"]
)

# Create Operating Margin
operating_margin = graph.add_calculation(
    name="OperatingMargin",
    input_names=["OperatingIncome", "Revenue"],
    operation_type="formula",
    formula="(operating_income / revenue) * 100",
    formula_variable_names=["operating_income", "revenue"]
)

# Calculate and display results for all periods
print("Calculated Financial Metrics:")
print("=" * 60)
for period in graph.periods:
    print(f"\n{period}:")
    print(f"  Revenue:           ${graph.calculate('Revenue', period):,.2f}")
    print(f"  COGS:              ${graph.calculate('COGS', period):,.2f}")
    print(f"  Gross Profit:      ${graph.calculate('GrossProfit', period):,.2f}")
    print(f"  Gross Margin:      {graph.calculate('GrossProfitMargin', period):.1f}%")
    print(f"  Operating Income:  ${graph.calculate('OperatingIncome', period):,.2f}")
    print(f"  Operating Margin:  {graph.calculate('OperatingMargin', period):.1f}%")


Calculated Financial Metrics:

2021:
  Revenue:           $1,000.00
  COGS:              $600.00
  Gross Profit:      $400.00
  Gross Margin:      40.0%
  Operating Income:  $200.00
  Operating Margin:  20.0%

2022:
  Revenue:           $1,200.00
  COGS:              $700.00
  Gross Profit:      $500.00
  Gross Margin:      41.7%
  Operating Income:  $260.00
  Operating Margin:  21.7%

2023:
  Revenue:           $1,400.00
  COGS:              $800.00
  Gross Profit:      $600.00
  Gross Margin:      42.9%
  Operating Income:  $320.00
  Operating Margin:  22.9%


## 4. Working with Built-in Metrics

The library includes a comprehensive metrics registry with 75+ financial metrics. Let's add some balance sheet items and calculate metrics.


In [27]:
# Add some balance sheet items for metric calculations
current_assets = graph.add_financial_statement_item(
    "CurrentAssets",
    {"2021": 500.0, "2022": 600.0, "2023": 700.0}
)

current_liabilities = graph.add_financial_statement_item(
    "CurrentLiabilities", 
    {"2021": 300.0, "2022": 350.0, "2023": 400.0}
)

total_assets = graph.add_financial_statement_item(
    "TotalAssets",
    {"2021": 2000.0, "2022": 2400.0, "2023": 2800.0}
)

total_equity = graph.add_financial_statement_item(
    "TotalEquity",
    {"2021": 1200.0, "2022": 1400.0, "2023": 1600.0}
)

# List some available metrics
print("Sample of available metrics:")
available_metrics = metric_registry.list_metrics()
for metric in available_metrics[:5]:
    print(f"  - {metric}")
print(f"  ... and {len(available_metrics) - 5} more metrics\n")

# Create standard financial ratios using the metric registry
# Note: We need to create a net_income node for ROA calculation
# Using formula calculation to apply tax rate
net_income = graph.add_calculation(
    name="NetIncome",
    input_names=["OperatingIncome"],
    operation_type="formula",
    formula="operating_income * 0.7",  # Assuming 30% tax rate
    formula_variable_names=["operating_income"]
)

# Add metric nodes to the graph
current_ratio = graph.add_metric(
    "current_ratio",
    input_node_map={
        "current_assets": "CurrentAssets",
        "current_liabilities": "CurrentLiabilities"
    }
)

# Calculate Return on Assets using direct metric calculation
data_nodes = {
    "net_income": graph.nodes["NetIncome"],
    "total_assets": graph.nodes["TotalAssets"]
}

print("Financial Ratios for 2023:")
print("-" * 40)

# Current Ratio
cr_value = graph.calculate("current_ratio", "2023")
print(f"Current Ratio: {cr_value:.2f}")

# Get metric definition and interpret the value
cr_metric_def = metric_registry.get("current_ratio")
cr_analysis = interpret_metric(cr_metric_def, cr_value)
print(f"  Rating: {cr_analysis['rating']}")
print(f"  Interpretation: {cr_analysis['interpretation_message']}")

# Return on Assets
roa_value = calculate_metric("return_on_assets", data_nodes, "2023")
print(f"\nReturn on Assets: {roa_value:.1f}%")

# Interpret ROA
roa_metric_def = metric_registry.get("return_on_assets")
roa_analysis = interpret_metric(roa_metric_def, roa_value)
print(f"  Rating: {roa_analysis['rating']}")
print(f"  Interpretation: {roa_analysis['interpretation_message']}")


Sample of available metrics:
  - adjusted_funds_from_operations
  - affo_multiple
  - affo_per_share
  - allowance_to_loans_ratio
  - altman_z"_score_(non_manufacturing)
  ... and 116 more metrics

Financial Ratios for 2023:
----------------------------------------
Current Ratio: 1.75
  Rating: good
  Interpretation: Good performance: 1.75

Return on Assets: 8.0%
  Rating: good
  Interpretation: Good performance: 8.00


## 5. Adjustments and Scenario Analysis

The adjustments system allows for what-if analysis and scenario modeling by applying changes to node values.


In [28]:
# Base case values
print("Base Case Analysis (2023):")
print(f"  Revenue: ${graph.calculate('Revenue', '2023'):,.2f}")
print(f"  Operating Income: ${graph.calculate('OperatingIncome', '2023'):,.2f}")
print(f"  Operating Margin: {graph.calculate('OperatingMargin', '2023'):.1f}%")


# Add a bullish scenario adjustment - 10% revenue increase
adj1 = graph.add_adjustment(
    node_name="Revenue",
    period="2023",
    value=1.10,  # 10% increase
    adj_type=AdjustmentType.MULTIPLICATIVE,
    reason="Bullish scenario - strong market demand",
    scenario="bullish",
    tags={"Scenario/Bullish", "Revenue"}
)

# Add a cost reduction adjustment
adj2 = graph.add_adjustment(
    node_name="OperatingExpenses",
    period="2023", 
    value=0.95,  # 5% reduction
    adj_type=AdjustmentType.MULTIPLICATIVE,
    reason="Cost optimization initiative",
    scenario="bullish",
    tags={"Scenario/Bullish", "CostReduction"}
)

# Get adjusted values for bullish scenario - Use AdjustmentFilter to specify scenario
print("\nBullish Scenario (2023):")
bullish_filter = AdjustmentFilter(include_scenarios={"bullish"})
bullish_revenue = graph.get_adjusted_value("Revenue", "2023", filter_input=bullish_filter)
print(f"  Adjusted Revenue: ${bullish_revenue:,.2f} (+10%)")

# Recalculate downstream metrics with adjustments
# Note: For a full scenario analysis, we'd need to propagate adjustments through the graph
# For now, let's show the impact manually
base_opex = graph.calculate("OperatingExpenses", "2023")
adjusted_opex = graph.get_adjusted_value("OperatingExpenses", "2023", filter_input=bullish_filter)
adjusted_gross_profit = bullish_revenue - graph.calculate("COGS", "2023")
adjusted_operating_income = adjusted_gross_profit - adjusted_opex
adjusted_operating_margin = (adjusted_operating_income / bullish_revenue) * 100

print(f"  Adjusted Operating Expenses: ${adjusted_opex:,.2f} (-5%)")
print(f"  Adjusted Operating Income: ${adjusted_operating_income:,.2f}")
print(f"  Adjusted Operating Margin: {adjusted_operating_margin:.1f}%")

# Add a bearish scenario
adj3 = graph.add_adjustment(
    node_name="Revenue",
    period="2023",
    value=0.85,  # 15% decrease
    adj_type=AdjustmentType.MULTIPLICATIVE,
    reason="Bearish scenario - economic downturn",
    scenario="bearish",
    tags={"Scenario/Bearish", "Revenue"}
)

# Compare scenarios
print("\nScenario Comparison (2023 Revenue):")
base_revenue = graph.calculate("Revenue", "2023")
bearish_filter = AdjustmentFilter(include_scenarios={"bearish"})
bearish_revenue = graph.get_adjusted_value("Revenue", "2023", filter_input=bearish_filter)

print(f"  Base Case:     ${base_revenue:,.2f}")
print(f"  Bullish (+10%): ${bullish_revenue:,.2f}")
print(f"  Bearish (-15%): ${bearish_revenue:,.2f}")

# List all adjustments
print("\nAll Adjustments:")
all_adjustments = graph.list_all_adjustments()
for adj in all_adjustments:
    print(f"  - {adj.node_name} ({adj.period}): {adj.type.value} {adj.value} - {adj.reason}")


Base Case Analysis (2023):
  Revenue: $1,400.00
  Operating Income: $320.00
  Operating Margin: 22.9%

Bullish Scenario (2023):
  Adjusted Revenue: $1,540.00 (+10%)
  Adjusted Operating Expenses: $266.00 (-5%)
  Adjusted Operating Income: $474.00
  Adjusted Operating Margin: 30.8%

Scenario Comparison (2023 Revenue):
  Base Case:     $1,400.00
  Bullish (+10%): $1,540.00
  Bearish (-15%): $1,190.00

All Adjustments:
  - Revenue (2023): multiplicative 1.1 - Bullish scenario - strong market demand
  - OperatingExpenses (2023): multiplicative 0.95 - Cost optimization initiative
  - Revenue (2023): multiplicative 0.85 - Bearish scenario - economic downturn


### Alternative Approach: Using Default Scenario

If you prefer not to import `AdjustmentFilter`, you can create adjustments without specifying a scenario (they'll use the default scenario) and then filter by tags:


In [29]:
# Alternative approach: Create adjustments without specifying scenario
# They will use the default scenario

# Add optimistic adjustments
adj_opt_revenue = graph.add_adjustment(
    node_name="Revenue",
    period="2023",
    value=1.15,  # 15% increase
    adj_type=AdjustmentType.MULTIPLICATIVE,
    reason="Optimistic scenario - market expansion",
    # No scenario parameter - uses default
    tags={"Optimistic", "Revenue"}
)

adj_opt_costs = graph.add_adjustment(
    node_name="COGS",
    period="2023",
    value=0.92,  # 8% reduction through efficiency gains
    adj_type=AdjustmentType.MULTIPLICATIVE,
    reason="Supply chain optimization",
    tags={"Optimistic", "CostReduction"}
)

# Now you can filter by tags and it will work with default scenario
print("Optimistic Scenario (using default scenario + tags):")
optimistic_revenue = graph.get_adjusted_value("Revenue", "2023", filter_input={"Optimistic"})
optimistic_cogs = graph.get_adjusted_value("COGS", "2023", filter_input={"Optimistic"})
optimistic_gross_profit = optimistic_revenue - optimistic_cogs
optimistic_gp_margin = (optimistic_gross_profit / optimistic_revenue) * 100

print(f"  Adjusted Revenue: ${optimistic_revenue:,.2f} (+15%)")
print(f"  Adjusted COGS: ${optimistic_cogs:,.2f} (-8%)")
print(f"  Adjusted Gross Profit: ${optimistic_gross_profit:,.2f}")
print(f"  Adjusted Gross Margin: {optimistic_gp_margin:.1f}%")

# Compare to base case
base_gp_margin = graph.calculate("GrossProfitMargin", "2023")
print(f"\nMargin improvement: {optimistic_gp_margin - base_gp_margin:.1f} percentage points")


Optimistic Scenario (using default scenario + tags):
  Adjusted Revenue: $1,610.00 (+15%)
  Adjusted COGS: $736.00 (-8%)
  Adjusted Gross Profit: $874.00
  Adjusted Gross Margin: 54.3%

Margin improvement: 11.4 percentage points
